In [189]:
import urllib.request
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [190]:
urls = ["http://killedbypolice.net", "http://www.killedbypolice.net/kbp2017", "http://www.killedbypolice.net/kbp2016", "http://www.killedbypolice.net/kbp2015", "http://www.killedbypolice.net/kbp2014", "http://www.killedbypolice.net/kbp2013"]

In [191]:
killedbypolice = pd.DataFrame(columns = ["date", "state", "gender", "race", "name", "age", "manner of death"])

In [192]:
for url in urls:
    # turn page into beautiful soup HTML
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    
    # all of the information is in one "row", so split it up
    table = soup.find_all("tr")[3].text
    rows = table.split("\n")
    
    # parse information into columns
    datecol = []
    statecol = []
    genracecol = []
    namecol = []
    agecol = []
    mannerdeathcol = []
    for row in rows:
        idnum = ""
        if "(" in row:
            idnum = row[row.find("(")+1 : row.find(")")]
            row = row[row.find(")")+2:]
        date = ""
        for i in range(len(row)):
            if i!=0 and (row[i].isupper() or row[i]=="\t"):
                date = row[0:i]
                row = row[i:]
                break
        state = ""
        genderrace = ""
        for i in range(len(row)):
            if row[i].islower():
                attrs = row[0:i-1].strip()
                state = attrs[0:2]
                genderrace = attrs[2:]
                row = row[i-1:]
                break
        name = row[:row.find(",")]
        try:
            age = int(row[row.find(",")+2:row.find("\t")])
        except:
            continue

        row = row[row.find("\t")+1:]
        mannerdeath = row[0]

        datecol.append(date)
        statecol.append(state)
        genracecol.append(genderrace)
        namecol.append(name)
        agecol.append(age)
        mannerdeathcol.append(mannerdeath)
    datecol = np.array(datecol)
    statecol = np.array(statecol)
    gendercol = []
    racecol = []
    for entry in genracecol:
        gendercol.append(entry[0])
        if "/" in entry:
            racecol.append(entry[2])
        else:
            racecol.append("")
    gendercol = np.array(gendercol)
    racecol = np.array(racecol)
    namecol = np.array(namecol)
    agecol = np.array(agecol)
    deaths = {"G": "shot",
             "T": "tasered",
             "R": "physical force",
             "C": "chemical",
             "V": "vehicle",
              "O": "other",
             "f": "",
             "\t": "",
             " ": "",
             '"': ""}
    mannerdeathcol = [entry.replace(entry, deaths[entry]) for entry in mannerdeathcol]
    mannerdeathcol = np.array(mannerdeathcol)
    
    onedf = pd.DataFrame({"date": datecol,
                                  "state": statecol,
                                  "gender": gendercol,
                                   "race": racecol,
                                  "name": namecol,
                                   "age": agecol,
                                  "manner of death": mannerdeathcol})
    killedbypolice = killedbypolice.append(onedf, ignore_index = True)

In [193]:
killedbypolice

,age,date,gender,manner of death,name,race,state
0,25,"April 21, 2018",M,shot,Terrance Carlton,,TN
1,48,"April 20, 2018",M,shot,James Bauduy,,FL
2,40,"April 19, 2018",M,shot,Charles Whitley,,KY
3,45,"April 18, 2018",M,shot,William Ray Simcoe Sr.,W,MO
4,48,"April 18, 2018",M,shot,David Gino Teneyuque,,MI
5,32,"April 18, 2018",M,shot,Delorean Pikyavit,I,UT
6,32,"April 18, 2018",M,shot,Justin Oakes,W,GA
7,25,"April 17, 2018",M,shot,Sanchez Lowe,,GA
8,39,"April 17, 2018",M,shot,Justin Monjay,,OK
9,41,"April 17, 2018",M,shot,Lonnie Marcel Bowen,B,UT


In [183]:
killedbypolice.loc[killedbypolice["name"]=="Justin Monjay"]

,age,date,gender,manner of death,name,race,state
7,39,"April 17, 2018",M,shot,Justin Monjay,,OK


In [197]:
dates = killedbypolice["date"]
months = [date[:date.find(" ")] for date in dates]
killedbypolice["month"] = months
years = [date[date.find(",")+2:] for date in dates]
killedbypolice["year"] = years

In [198]:
killedbypolice

,age,date,gender,manner of death,name,race,state,month,year
0,25,"April 21, 2018",M,shot,Terrance Carlton,,TN,April,2018
1,48,"April 20, 2018",M,shot,James Bauduy,,FL,April,2018
2,40,"April 19, 2018",M,shot,Charles Whitley,,KY,April,2018
3,45,"April 18, 2018",M,shot,William Ray Simcoe Sr.,W,MO,April,2018
4,48,"April 18, 2018",M,shot,David Gino Teneyuque,,MI,April,2018
5,32,"April 18, 2018",M,shot,Delorean Pikyavit,I,UT,April,2018
6,32,"April 18, 2018",M,shot,Justin Oakes,W,GA,April,2018
7,25,"April 17, 2018",M,shot,Sanchez Lowe,,GA,April,2018
8,39,"April 17, 2018",M,shot,Justin Monjay,,OK,April,2018
9,41,"April 17, 2018",M,shot,Lonnie Marcel Bowen,B,UT,April,2018


In [199]:
killedbypolice.to_csv("killedbypolice.csv", index=False)